<a href="https://colab.research.google.com/github/Maks3174/Phy/blob/main/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B0_%D0%BD%D0%B5%D0%B9%D1%80%D0%BE%D0%BC%D0%B5%D1%80%D0%B5%D0%B6%D1%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Імпорт модулів

In [15]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F

Дані про морське вухо(молюск)

<div>
<img src="https://m.media-amazon.com/images/I/81-FC5TlLtL.jpg" width="500"/>
</div>

In [16]:
df = pd.read_csv('https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module3/data/abalone.csv')
df.drop(columns=['Type', 'WholeWeight', 'ShuckedWeight', 'VisceraWeight', 'ShellWeight'], inplace=True)

In [17]:
df.head()

,LongestShell,Diameter,Height,Rings
0,0.455,0.365,0.095,15
1,0.350,0.265,0.090,7
2,0.530,0.420,0.135,9
3,0.440,0.365,0.125,10
4,0.330,0.255,0.080,7


Потрібно на основі розмірів молюска спрогнозувати кількість кілець в мушлі `Rings`. Використовуйте процесор cpu

# Завдання 1
Створіть клас Dataset

In [18]:
class AbaloneDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe
        self.features = dataframe[['LongestShell', 'Diameter', 'Height']].values
        self.labels = dataframe['Rings'].values

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = torch.tensor(self.features[idx], dtype=torch.float32)
        y = torch.tensor(self.labels[idx], dtype=torch.float32)
        return x, y

df = pd.read_csv('https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module3/data/abalone.csv')
df.drop(columns=['Type', 'WholeWeight', 'ShuckedWeight', 'VisceraWeight', 'ShellWeight'], inplace=True)

dataset = AbaloneDataset(df)
print(f'Dataset length: {len(dataset)}')

Dataset length: 4177


# Завдання 2
Створіть DataLoader. Перевірте розміри даних в першому пакеті. Розмір пакетів оберіть самостійно

In [19]:
batch_size = 32
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

for batch in dataloader:
    features, labels = batch
    print(f'Feature batch shape: {features.shape}')
    print(f'Label batch shape: {labels.shape}')
    break

Feature batch shape: torch.Size([32, 3])
Label batch shape: torch.Size([32])


# Завдання 3
Створіть нейромережу:
* Кількість шарів оберіть між 1-3
* Виберіть різні функції активації: Relu, LeakyRelu, Swish
* Кілкість нейронів у прихованих шарах повинна **не зростати**

In [11]:
class AbaloneNet(nn.Module):
    def __init__(self):
        super(AbaloneNet, self).__init__()
        self.fc1 = nn.Linear(3, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = AbaloneNet()
print(model)

AbaloneNet(
  (fc1): Linear(in_features=3, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=1, bias=True)
)


# Завдання 4
Визначте функцію втрат та оптимізатор

In [12]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Завдання 5
Натренуйте нейромережу. візміть невелику кількість епох

In [13]:
num_epochs = 10

for epoch in range(num_epochs):
    for batch in dataloader:
        features, labels = batch

        outputs = model(features)
        loss = criterion(outputs.squeeze(), labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/10], Loss: 10.1895
Epoch [2/10], Loss: 3.3664
Epoch [3/10], Loss: 3.8877
Epoch [4/10], Loss: 2.0455
Epoch [5/10], Loss: 3.6857
Epoch [6/10], Loss: 8.2756
Epoch [7/10], Loss: 4.5502
Epoch [8/10], Loss: 12.6966
Epoch [9/10], Loss: 6.3212
Epoch [10/10], Loss: 7.1869


# Завдання 6
Збережіть нейромережу та зробіть прогноз

In [14]:
torch.save(model.state_dict(), 'abalone_model.pth')

model.load_state_dict(torch.load('abalone_model.pth'))
model.eval()

sample = torch.tensor([0.455, 0.365, 0.095], dtype=torch.float32)
with torch.no_grad():
    prediction = model(sample)
    print(f'Predicted Rings: {prediction.item()}')

Predicted Rings: 9.048530578613281
